# DAUM / NAVER

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pymongo import MongoClient

import datetime
from datetime import timedelta
import random
import hashlib
import collections

style.use('ggplot')
%matplotlib inline
keywords = ['이해찬','초록우산어린이재단', '산업혁명', '강원도', '신한은행', '상견례', '춘천시장', '카카오커머스', '초록우산', '오징어게임', 'sk가스', '블랙핑크', '4차산업혁명위원회', '오대훈', '디카르고', '행복', '최문순', '원주시', '맨유', 'mz세대', '아이폰13', '아스트라', '야구', '도이치모터스', '한국전력', '국세청', '20대', '오징어', '출마', '무소속', '위하고', '월드시리즈', '영림원', 'EY한영', '할인', '얀센', '대선', '티맥스', '나경원', '카카오커머스병합', '소나기', '빅뱅', '국민의힘', '디피', '산업단지관리공단', '누누', '솔샤르', '안철수', '육군', 'minsriver', '한국전력공사', '정세균', '이재수', '중앙지검', '오세훈', '더존비즈온', '백신', '모더나', '이낙연', '마스크', '로지스팟', '녹십자홀딩스', '매출채권팩토링', '광주', '이명박', '비트코인', '강원대학교', '이벤트', '개리', '탬파베이', '얀센백신', '지원금', 'mlb', '홍준표', '맨체스터유나이티드', '콘테', '황상무', '무지개', '삼성라이온즈', '화천대유', '정책', '게임', '유승민', '삼일회계법인', '이광재', '제주도', '키워드', '권지용', '여행', '테스트', '강릉', '윤희숙', '화이자백신', '문재인', '바베큐', '더존', 'GC녹십자홀딩스', '창원시', '민주당', '강민', '배구', '이재명', '코로나', '춘천', '윤석열', '4차위', '부산', '웹케시', '송영길', '애틀란타브레이브스', '노트북', '메타버스', '겨울', '축구', '공약', '김인규', '리버풀', '현대백화점', '김연경', '사천', '월드컵', '카카오', '한림대학교', '날씨', '공정', '정윤환', '야구장', '김용택', 'NFT', '트위터', '화이자', '향수', 'ERP']

client = MongoClient('localhost', 27017, username='mongo', password='mongo')
db = client.news
collection = db.related_keyword



naver_dictionary = {}
daum_dictionary = {}

def naver_crawling_related_word(word_list, n):
    if n==0:
        print("크롤링 종료")
    else:
        merge_df = pd.DataFrame()

        for i in range(len(word_list)):
            temp_df = pd.DataFrame()
            temp_list = []

            url = f'https://search.naver.com/search.naver?where=nexearch&query={word_list[i]}'

            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')

            ul_inner = soup.find('ul', {'class':'lst_related_srch _list_box'})

            try:
                keyword_list = soup.find('ul', {'class':'lst_related_srch _list_box'}).find_all('li')
            except:
                if pd.isna(ul_inner):
                    try:
                        keyword_list = soup.find('ul', {'class':'lst_related_srch _list_box'}).find_all('li')
                    except:
                        keyword_list = []

            # span_list 가 공백일 때 예외 처리
            if len(keyword_list) == 0:
                pass
            else:
                for k, word in enumerate(keyword_list):
                    if len(keyword_list[k].text.replace(word_list[i],'').strip()) == 1:
                        temp_list.append(keyword_list[k].text.strip())
                    else:
                          temp_list.append(keyword_list[k].text.strip())

                temp_df['sub_word'] = temp_list
                temp_df['sup_word'] = word_list[i]
                temp_df = pd.DataFrame(temp_df, columns=['sup_word', 'sub_word'])

                merge_df = pd.concat([merge_df, temp_df], axis=0)
        naver_dictionary[n] = merge_df
        # 재귀호출
        pass_word_list = merge_df['sub_word'].values

        print('네이버 크롤링 시작')
        naver_crawling_related_word(pass_word_list, n-1)

def daum_crawling_related_word(word_list, n):
    if n==0:
        print("크롤링 종료")
    else:
        merge_df = pd.DataFrame()

        for i in range(len(word_list)):
            temp_df = pd.DataFrame()
            temp_list = []

            url = f'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q={word_list[i]}'

            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')

            div_inner = soup.find('div', {'class':'list_keyword type2'})

            try:
                div_inner = soup.find('div', {'class':'list_keyword type2'})
                span_list = div_inner.find_all('span', {'class':'wsn'})
            except:
                if pd.isna(div_inner):
                    try:
                        div_inner = soup.find('div', {'class':'list_keyword'})
                        span_list = div_inner.find_all('span', {'class':'wsn'})
                    except:
                        span_list = []

            # span_list 가 공백일 때 예외 처리
            if len(span_list) == 0:
                pass
            else:
                for k, word in enumerate(span_list):
                    if len(span_list[k].text.replace(word_list[i],'').strip()) == 1:
                        temp_list.append(span_list[k].text)
                    else:
                        temp_list.append(span_list[k].text)

                temp_df['sub_word'] = temp_list
                temp_df['sup_word'] = word_list[i]
                temp_df = pd.DataFrame(temp_df, columns=['sup_word', 'sub_word'])

                merge_df = pd.concat([merge_df, temp_df], axis=0)

        daum_dictionary[n] = merge_df

        # 재귀호출
        pass_word_list = merge_df['sub_word'].values
        print('다음 크롤링 시작')
        daum_crawling_related_word(pass_word_list, n-1)    

word_list = ['윤석열']
try:
    naver_crawling_related_word(word_list, 2)
    daum_crawling_related_word(word_list, 2)
except Exception as e:
    print(e)
    
    
daum_dictionary[2].columns = ['sup_word', 'sub_word1']
daum_dictionary[1].columns = ['sub_word1', 'sub_word2']

naver_dictionary[2].columns = ['sup_word', 'sub_word1']
naver_dictionary[1].columns = ['sub_word1', 'sub_word2']



'sub_word'


KeyError: 2

# Google 크롤링

In [2]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

def get_results(query):
    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.co.uk/search?q=" + query)
    
    return response

def parse_results(response):
    
    css_identifier_result = ".s75CSd"
    css_identifier_title = "b"
#     css_identifier_link = ".yuRUbf a"
#     css_identifier_text = ".IsZvec"
    
    results = response.html.find(css_identifier_result)

    output = []
    for result in results:
        text= result.find(css_identifier_title, first=True).text
        output.append(text)
    return output

def google_search(query):
    response = get_results(query)
    return parse_results(response)

keywords = ['윤석열']
for keyword in keywords:
    print(google_search(keyword))
    
go_li = google_search(keyword)
google_df = pd.DataFrame(go_li)
google_df['sup_word'] = keywords[0]
google_df.columns = ['sub_word1','sup_word']
google_df


['홍준표', '부인', '자녀', '군대', '김건희', '키', '전부인', '고향']


,sub_word1,sup_word
0,홍준표,윤석열
1,부인,윤석열
2,자녀,윤석열
3,군대,윤석열
4,김건희,윤석열
5,키,윤석열
6,전부인,윤석열
7,고향,윤석열


In [22]:
result_df = pd.concat([daum_dictionary[2], naver_dictionary[2],google_df])
result_df = result_df.reset_index()
result_df = result_df.drop('index',1)

result_df

<ipython-input-22-62ebfd7791b6>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  result_df = result_df.drop('index',1)


,sup_word,sub_word1
0,더존비즈온,더존비즈온 주가
1,더존비즈온,더존비즈온 주가 전망
2,더존비즈온,펄어비스
3,더존비즈온,더존비즈온 연봉
4,더존비즈온,더존비즈온 실적
5,더존비즈온,더존비즈온 배당
6,더존비즈온,카카오
7,더존비즈온,더존비즈온 고객센터
8,더존비즈온,더존비즈온 채용
9,더존비즈온,더존비즈온 목표주가


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib 

import networkx as nx
import matplotlib.font_manager as fm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from matplotlib import style
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
import collections

style.use('ggplot')
%matplotlib inline


def netword_drawing(re_df):
    print("그리기 시작")
    G = nx.Graph()
    
    for sup in list(re_df['keyword'].values):
        G.add_node(sup)
        
        for sub1 in list(re_df['sub_word'].values):
            G.add_node(sub1)
            G.add_edge(sup, sub1)
            
#             for sub2 in list(set(sub_df2['keyword'].values)):
#                 G.add_node(sub2)
#                 G.add_edge(sub1, sub2)
                
    color_map = []
    for node in G:
        if node == re_df['keyword'].values[0]:
            color_map.append('red')
        else:
            color_map.append('green')      
    
    plt.figure(figsize=(20,20))
#     d = dict(G.degree)
    pos=nx.shell_layout(G)
    d = dict(zip(re_df['sub_word'], re_df['views']))

    nx.draw(G, nodelist=list(d), node_size=[int(v) for v in d.values()], arrowstyle='->',arrowsize=20, width=3, edge_cmap=plt.cm.OrRd,font_family='AppleGothic', font_size=20, node_color = color_map, with_labels = True, font_weight='bold')

    plt.axis('off')
#     plt.savefig('networx_picture_'+total_words['sup_word'].values[0]+'.png')
    plt.show()

netword_drawing(re_df)

In [23]:
import warnings
warnings.filterwarnings(action='ignore')

import matplotlib 

import networkx as nx
import matplotlib.font_manager as fm


def netword_drawing(total_words):
    print("그리기 시작")
    G = nx.Graph()
    
    for sup in list(set(total_words['sup_word'].values)):
        G.add_node(sup)
        temp_df1 = total_words[total_words['sup_word']==sup]
        
        for sub1 in list(set(temp_df1['sub_word1'].values)):
            G.add_node(sub1)
            G.add_edge(sup, sub1)
            temp_df2 = temp_df1[temp_df1['sub_word1']==sub1]
            
            for sub2 in list(set(temp_df2['sub_word2'].values)):
                G.add_node(sub2)
                G.add_edge(sub1, sub2)
                
    color_map = []
    for node in G:
        if node == total_df['sup_word'].values[0]:
            color_map.append('red')
        elif node in list(set(total_df['sub_word1'].values)):
            color_map.append('green')
        else:
            color_map.append('blue')
    
    plt.figure(figsize=(100,100))
    d = dict(G.degree)
    pos=nx.shell_layout(G)

    nx.draw(G, nodelist=d.keys(), node_size=[v*100 for v in d.values()], font_family='AppleGothic', font_size=20,node_color = color_map, with_labels = True)

    plt.axis('off')
    plt.savefig('networx_picture_'+total_words['sup_word'].values[0]+'.png')
    plt.show()
    


In [24]:
total_df = total_df.reset_index(drop=True)
total_df

,sup_word,sub_word1,sub_word2
0,더존비즈온,더존비즈온 주가,더존비즈온 주가 전망
1,더존비즈온,더존비즈온 주가,더존비즈온 목표주가
2,더존비즈온,더존비즈온 주가,케이엠더블유 주가
3,더존비즈온,더존비즈온 주가,비즈니스온 주가
4,더존비즈온,더존비즈온 주가,아시아나항공 주가
...,...,...,...
196,더존비즈온,더존비즈온 주가,리노공업 주가
197,더존비즈온,더존비즈온 주가,더존 주가
198,더존비즈온,더존비즈온 채용,더존비즈온
199,더존비즈온,더존비즈온 연봉,더존비즈온


In [25]:
netword_drawing(total_df.loc[:len(total_df)])

그리기 시작


# 키워드 언급량

네이버 검색 광고 API 기반

- !pip3 install powernad / jsonpickle
- https://github.com/devkingsejong/python-PowerNad

## 네이버 검색광고 API는 네이버 검색광고 상품 중 일부 상품(사이트검색광고)에 대한 기능 및 데이터를 제공합니다.

In [11]:
import hashlib
import hmac
import base64
from pymongo import MongoClient
from pprint import pprint
import requests
import time
import random
import requests
from powernad.API import RelKwdStat
import pandas as pd
BASE_URL = 'https://api.naver.com'
API_KEY = '0100000000f93df373446ea1ee430f49c14690f1f0ec1ae6a624fa652419ab52ef231273f0'
SECRET_KEY = 'AQAAAAD5PfNzRG6h7kMPScFGkPHwvuPYo+dmLIk2xT+T3eUgyg=='
CUSTOMER_ID = '2414867'

keywords = ['이해찬','초록우산어린이재단', '산업혁명', '강원도', '신한은행', '상견례', '춘천시장', '카카오커머스', '초록우산', '오징어게임', 'sk가스', '블랙핑크', '4차산업혁명위원회', '오대훈', '디카르고', '행복', '최문순', '원주시', '맨유', 'mz세대', '아이폰13', '아스트라', '야구', '도이치모터스', '한국전력', '국세청', '20대', '오징어', '출마', '무소속', '위하고', '월드시리즈', '영림원', 'EY한영', '할인', '얀센', '대선', '티맥스', '나경원', '카카오커머스병합', '소나기', '빅뱅', '국민의힘', '디피', '산업단지관리공단', '누누', '솔샤르', '안철수', '육군', 'minsriver', '한국전력공사', '정세균', '이재수', '중앙지검', '오세훈', '더존비즈온', '백신', '모더나', '이낙연', '마스크', '로지스팟', '녹십자홀딩스', '매출채권팩토링', '광주', '이명박', '비트코인', '강원대학교', '이벤트', '개리', '탬파베이', '얀센백신', '지원금', 'mlb', '홍준표', '맨체스터유나이티드', '콘테', '황상무', '무지개', '삼성라이온즈', '화천대유', '정책', '게임', '유승민', '삼일회계법인', '이광재', '제주도', '키워드', '권지용', '여행', '테스트', '강릉', '윤희숙', '화이자백신', '문재인', '바베큐', '더존', 'GC녹십자홀딩스', '창원시', '민주당', '강민', '배구', '이재명', '코로나', '춘천', '윤석열', '4차위', '부산', '웹케시', '송영길', '애틀란타브레이브스', '노트북', '메타버스', '겨울', '축구', '공약', '김인규', '리버풀', '현대백화점', '김연경', '사천', '월드컵', '카카오', '한림대학교', '날씨', '공정', '정윤환', '야구장', '김용택', 'NFT', '트위터', '화이자', '향수', 'ERP']

client = MongoClient('localhost', 27017, username='mongo', password='mongo')
db = client.news
collection = db.keyword_views_count

for keyword in keywords:
    data_list = []
    
    relKwdStat = RelKwdStat.RelKwdStat(BASE_URL, API_KEY, SECRET_KEY, CUSTOMER_ID)
    time.sleep(1)
    kwdDataList = relKwdStat.get_rel_kwd_stat_list(None, hintKeywords=keyword, showDetail='1')
    
    for outdata in kwdDataList:
        relKeyword = outdata.relKeyword # 연관 키워드
        monthleyPcQcCnt = outdata.monthlyPcQcCnt # 30일간 PC 조회수
        monthleyMobileQcCnt = outdata.monthlyMobileQcCnt # 30일간 모바일 조회수
        monthlyAveMobileClkCnt = outdata.monthlyAveMobileClkCnt # 4주간 평균 모바일 클릭수
        monthlyAvePcCtr = outdata.monthlyAvePcCtr # 4주간 평균 PC 클릭율
        monthlyAveMobileCtr = outdata.monthlyAveMobileCtr # 4주간 평균 모바일 클릭율
        plAvgDepth = outdata.plAvgDepth # 4주간 평균 PC 광고수
        compIdx = outdata.compIdx
        
        if monthleyPcQcCnt == '< 10':
            monthleyPcQcCnt = 0
        
        if monthleyMobileQcCnt == '< 10':
            monthleyMobileQcCnt = 0
            
        json_obj = {
            '키워드' : keyword,
            '연관 키워드':relKeyword,
            '월간 PC 검색수':monthleyPcQcCnt,
            '월간 Mobile 검색수':monthleyMobileQcCnt,
            '검색수합계': int(monthleyPcQcCnt)+int(monthleyMobileQcCnt),
            '월평균 PC 클릭수' : monthlyAvePcCtr,
            '월평균 Mobile 클릭수' : monthlyAveMobileClkCnt,
            '월평균 PC 클릭율': monthlyAvePcCtr,
            '월평균 Mobile 클릭율':monthlyAveMobileCtr,
            '경쟁정도':compIdx,
            '월평균노출광고수':plAvgDepth
        }
#         pprint(json_obj)
        try:
            collection.insert_one(json_obj)
            print(keyword)
        except Exception as e:
            print(e, keyword)

이해찬
초록우산어린이재단
산업혁명
강원도
신한은행
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례
상견례


야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
야구
도이치모터스


KeyboardInterrupt: 

In [5]:
naverKwdApi("삼성")

,키워드,연관 키워드,월간 PC 검색수,월간 Mobile 검색수,검색수합계,월평균 PC 클릭수,월평균 Mobile 클릭수,월평균 PC 클릭율,월평균 Mobile 클릭율,경쟁정도,월평균노출광고수
0,삼성,삼성,93700,210200,303900,0.07,978.0,0.07,0.51,중간,13
1,삼성,중고차,101400,405800,507200,2.05,18985.8,2.05,5.00,높음,15
2,삼성,스투시,26400,148100,174500,0.64,1166.7,0.64,0.81,높음,15
3,삼성,중고차매매사이트,21500,118800,140300,2.52,5044.4,2.52,4.56,높음,15
4,삼성,스투시반팔,300,3090,3390,0.67,49.5,0.67,1.61,높음,15
5,삼성,데우스엑스마키나,2730,11100,13830,1.52,272.0,1.52,2.57,높음,15
6,삼성,휴대용,460,1040,1500,0.07,1.3,0.07,0.14,높음,15
7,삼성,중고차사이트,6720,31900,38620,2.74,2564.4,2.74,8.50,높음,15
8,삼성,중고자동차,5910,17400,23310,3.42,1091.2,3.42,6.71,높음,15
9,삼성,칼하트,23500,97400,120900,0.37,578.6,0.37,0.61,높음,15


In [28]:
df = naverKwdApi("더존")
df

,키워드,연관 키워드,월간 PC 검색수,월간 Mobile 검색수,검색수합계,월평균 PC 클릭수,월평균 Mobile 클릭수,월평균 PC 클릭율,월평균 Mobile 클릭율,경쟁정도,월평균노출광고수
0,더존,더존,24200,8120,32320,0.18,4.0,0.18,0.06,높음,14
1,더존,더존프로그램,2840,2600,5440,0.13,19.7,0.13,0.84,높음,15
2,더존,더존아이큐브,3070,740,3810,0.10,0.3,0.10,0.05,높음,14
3,더존,더존교육,710,230,940,0.66,9.3,0.66,4.05,높음,15
4,더존,더존고객센터,1780,220,2000,0.02,0.0,0.02,0.00,중간,6
...,...,...,...,...,...,...,...,...,...,...,...
469,더존,공전소,70,70,140,1.02,0.0,1.02,0.00,높음,3
470,더존,공인전자문서센터,140,70,210,0.00,0.0,0.00,0.00,높음,3
471,더존,공인전자문서보관,0,0,0,0.00,0.0,0.00,0.00,낮음,0
472,더존,그룹웨어더존,10,0,10,2.74,0.0,2.74,0.00,중간,9
